## Problem 1

In [1]:
import tensorflow as tf

# Input Layer
inputData = tf.constant([[100,150]])

# Hidden Layer 1
W1 = tf.constant([[5,10,15,20],[25,30,35,40]])
b1 = tf.constant([[17,19,21,23]])

H1 = tf.matmul(inputData,W1)+b1
H1_Activation = H1 # Linear Activation

# Output Layer
W2 = tf.constant([[30,35],[40,45],[70,75],[80,85]])
b2 = tf.constant([[35,36]])

H2 = tf.matmul(H1_Activation,W2)+b2
H2_Activation = H2 # Linear Activation

with tf.Session() as sess:
    print('The values of outputs are',sess.run(H2_Activation))

The values of outputs are [[1464615 1587516]]


## Problem 2

In [2]:
import tensorflow as tf
import numpy as np

# Data
x_data = np.array([[0,0],[1,0],[0,1],[1,1]])
y_data = np.array([[0],[1],[1],[0]])

# Constants
learning_rate = 0.1
epochs = 100
n_input, n_hidden, n_output = 2,3,1

# Input Layer
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# Hidden Layer 1
W1 = tf.Variable([[-4.,-6.,-5.],[3.,6.,4.]],name='Weight1')
b1 = tf.Variable([[-2.,3.,-2.]],name='Bias1')

H1 = tf.matmul(X,W1)+b1
H1_Activation = tf.sigmoid(H1)

# Output Layer
W2 = tf.Variable([[5.],[-9.],[7.]],name='Weight2')
b2 = tf.Variable([[4.]],name='Bias2')

output = tf.matmul(H1_Activation,W2)+b2
output_Activation = tf.sigmoid(output)

# Cost Function and Optimizer
# Cost = Cross entropy
# Optimizer = Gradient Descent
cost = tf.reduce_mean(-Y*tf.log(output_Activation)-(1-Y)*tf.log(1-output_Activation))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initialize all variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for step in range(epochs):
        sess.run(optimizer,feed_dict={X:x_data,Y:y_data})
        
        if step%10 == 0:
            print("Epoch:%d, cost:%.5f"%(step,sess.run(cost,feed_dict={X:x_data,Y:y_data})))
            #print(H1_Activation)
    
    for i in range(len(y_data)):
        err = tf.reduce_mean(tf.squared_difference(
            tf.cast(y_data[i],tf.float32),
            sess.run(output_Activation,feed_dict={X:x_data[i:i+1]})))
        
        print()
        print('The input is ',x_data[i])
        print('The true output is ',y_data[i])
        print('The computed output is ',sess.run(tf.reduce_mean(
            output_Activation),feed_dict={X:x_data[i:i+1]}))
        print('The squared error is ',sess.run(err))

Epoch:0, cost:0.02385
Epoch:10, cost:0.02360
Epoch:20, cost:0.02336
Epoch:30, cost:0.02314
Epoch:40, cost:0.02294
Epoch:50, cost:0.02275
Epoch:60, cost:0.02257
Epoch:70, cost:0.02241
Epoch:80, cost:0.02225
Epoch:90, cost:0.02211

The input is  [0 0]
The true output is  [0]
The computed output is  0.033769593
The squared error is  0.0011403854

The input is  [1 0]
The true output is  [1]
The computed output is  0.971751
The squared error is  0.0007980074

The input is  [0 1]
The true output is  [1]
The computed output is  0.99025947
The squared error is  9.4877956e-05

The input is  [1 1]
The true output is  [0]
The computed output is  0.015018627
The squared error is  0.00022555915


## Problem 3

In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

RANDOM_SEED = 42
tf.set_random_seed(RANDOM_SEED)

In [4]:
# Features data preprocessing
data = pd.read_csv('Admissions.csv')

features = data[['gre','gpa','rank']]
featuresScale = preprocessing.minmax_scale(features)

print('Original features\n',features[0:5])
print()
print('Scaled features\n',featuresScale[0:5])

Original features
    gre   gpa  rank
0  380  3.61     3
1  660  3.67     3
2  800  4.00     1
3  640  3.19     4
4  520  2.93     4

Scaled features
 [[0.27586207 0.77586207 0.66666667]
 [0.75862069 0.81034483 0.66666667]
 [1.         1.         0.        ]
 [0.72413793 0.53448276 1.        ]
 [0.51724138 0.38505747 1.        ]]


/Users/shijizhao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64.
  """


In [5]:
# Label Preprocessing
labels = data['admit']

one_hot = np.zeros(shape=(len(labels),2))
for i in range(len(labels)):
    # [1,0] means admitted, [0,1] means not admitted
    one_hot[i,(1-labels[i])]=1

print('Original labels\n',labels[0:5])
print()
print('One hot labels\n',one_hot[0:5])

Original labels
 0    0
1    1
2    1
3    1
4    0
Name: admit, dtype: int64

One hot labels
 [[0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]]


In [6]:
# Split dataset into training and testing
train_X, test_X, train_Y, test_Y = train_test_split(
    featuresScale, one_hot, test_size=0.3, random_state=RANDOM_SEED)

In [7]:
# Constants
learning_rate = 0.01
epochs = 101
n_input, n_output = 3,2
parameter_set = []   # Used to save weights and biases in each run

for n_hidden in range(1,11):
    # Reset the graph before next run
    tf.reset_default_graph()
    
    # Input Layer
    X = tf.placeholder(tf.float32)
    Y = tf.placeholder(tf.float32)
    
    # Hidden Layer 1
    W1 = tf.Variable(tf.random_uniform([n_input,n_hidden],-1.0,1.0),name='Weight1')
    b1 = tf.Variable(tf.zeros([n_hidden]),name='Bias1')
    
    H1 = tf.matmul(X,W1)+b1
    H1_Activation = tf.nn.relu(H1)
    
    # Output Layer
    W2 = tf.Variable(tf.random_uniform([n_hidden,n_output],-1.0,1.0),name='Weight2')
    b2 = tf.Variable(tf.zeros([n_output]),name='Bias2')
    
    output = tf.matmul(H1_Activation,W2)+b2
    predict = tf.argmax(output,axis=1)
    
    # Cost Function and Optimizer
    # Cost = Cross entropy
    # Optimizer = Gradient Descent
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y,logits=output))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
    
    # Initialize all variables
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        print()
        print('Nodes number in hidden layer: ',n_hidden)
        sess.run(init)
        
        for step in range(epochs):
            cst_all = 0
            for i in range(len(train_X)):
                [op,cst] = sess.run([optimizer,cost],
                                     feed_dict={X:train_X[i:i+20],Y:train_Y[i:i+20]})
                
                cst_all += cst
                
            cst_all = cst_all/len(train_X)
            test_accuracy = np.mean(np.argmax(test_Y,axis=1)==
                                   sess.run(predict,feed_dict={X:test_X}))
            
            
            if step%50 == 0:
                print("Epoch:%d, acc:%.5f, cost:%.5f"
                      %(step, test_accuracy, cst_all))
        
        tvars = tf.trainable_variables()
        tvars_vals = sess.run(tvars)
        
        # Save the name of the variable alongside its value in a dictionary
        parameters = {var.name:val for var, val in zip(tvars, tvars_vals)}
        parameter_set.append(parameters)


Nodes number in hidden layer:  1
Epoch:0, acc:0.67500, cost:0.64902
Epoch:50, acc:0.70833, cost:0.57035
Epoch:100, acc:0.71667, cost:0.57007

Nodes number in hidden layer:  2
Epoch:0, acc:0.67500, cost:0.65350
Epoch:50, acc:0.67500, cost:0.62157
Epoch:100, acc:0.67500, cost:0.62157

Nodes number in hidden layer:  3
Epoch:0, acc:0.67500, cost:0.65879
Epoch:50, acc:0.71667, cost:0.56774
Epoch:100, acc:0.71667, cost:0.56701

Nodes number in hidden layer:  4
Epoch:0, acc:0.67500, cost:0.64994
Epoch:50, acc:0.73333, cost:0.56987
Epoch:100, acc:0.71667, cost:0.56786

Nodes number in hidden layer:  5
Epoch:0, acc:0.67500, cost:0.64921
Epoch:50, acc:0.71667, cost:0.57014
Epoch:100, acc:0.71667, cost:0.56982

Nodes number in hidden layer:  6
Epoch:0, acc:0.67500, cost:0.61641
Epoch:50, acc:0.70833, cost:0.56281
Epoch:100, acc:0.71667, cost:0.56081

Nodes number in hidden layer:  7
Epoch:0, acc:0.68333, cost:0.64137
Epoch:50, acc:0.74167, cost:0.56650
Epoch:100, acc:0.72500, cost:0.56277

Nodes

We can see that the highest accuracy occurs when n_hidden = 7. The weights and biases of that neural network model are as follows:

In [8]:
for key,val in parameter_set[6].items():
    print('%s\n'%key,val)

Weight1:0
 [[-0.4418435   0.7490924   1.129238   -0.06484197 -0.7350185   0.82714206
   0.28770313]
 [-0.87327003 -1.0241653   0.0476774   0.6508952  -1.0116814  -0.9289226
  -0.9616844 ]
 [ 0.25594115 -0.99001855 -0.64674044 -0.84080595  1.1995713  -0.25539625
   0.1302991 ]]
Bias1:0
 [ 0.         -0.04817996 -0.14377616  0.15156877  0.17747189  0.04525077
  0.77388877]
Weight2:0
 [[-0.623121    0.31207705]
 [ 1.1367003  -0.86709774]
 [ 1.1714671   0.2767058 ]
 [ 0.15580466 -0.96104336]
 [-0.37090987  1.1774316 ]
 [ 0.8475938   0.20807974]
 [-0.78055274  0.30731335]]
Bias2:0
 [-0.40363762  0.40363762]


## Problem 4

In [9]:
import tensorflow as tf
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

RANDOM_SEED = 42
tf.set_random_seed(RANDOM_SEED)

In [10]:
# Features data preprocessing
data = pd.read_csv('Advertising.csv')

features = data[['TV','Radio','Newspaper']]
labels = data['Sales']
featuresScale = preprocessing.minmax_scale(features)

print('Original features\n',features[0:5])
print()
print('Scaled features\n',featuresScale[0:5])
print()
print('Labels\n',labels[0:5])

Original features
       TV  Radio  Newspaper
0  230.1   37.8       69.2
1   44.5   39.3       45.1
2   17.2   45.9       69.3
3  151.5   41.3       58.5
4  180.8   10.8       58.4

Scaled features
 [[0.77578627 0.76209677 0.60598065]
 [0.1481231  0.79233871 0.39401935]
 [0.0557998  0.92540323 0.60686016]
 [0.50997633 0.83266129 0.51187335]
 [0.60906324 0.21774194 0.51099384]]

Labels
 0    22.1
1    10.4
2     9.3
3    18.5
4    12.9
Name: Sales, dtype: float64


In [11]:
# Split dataset into training and testing
train_X, test_X, train_Y, test_Y = train_test_split(
    featuresScale, labels, test_size=0.3, random_state=RANDOM_SEED)

In [12]:
# Constants
learning_rate = 0.01
epochs = 101
n_input, n_output = 3,1
parameter_set = []   # Used to save weights and biases in each run

for n_hidden in range(1,11):
    # Reset the graph before next run
    tf.reset_default_graph()
    
    # Input Layer
    X = tf.placeholder(tf.float32)
    Y = tf.placeholder(tf.float32)
    
    # Hidden Layer 1
    W1 = tf.Variable(tf.random_uniform([n_input,n_hidden],-1.0,1.0),name='Weight1')
    b1 = tf.Variable(tf.zeros([n_hidden]),name='Bias1')
    
    H1 = tf.matmul(X,W1)+b1
    H1_Activation = tf.nn.relu(H1)
    
    # Output Layer
    W2 = tf.Variable(tf.random_uniform([n_hidden,n_output],-1.0,1.0),name='Weight2')
    b2 = tf.Variable(tf.zeros([n_output]),name='Bias2')
    
    output = tf.matmul(H1_Activation,W2)+b2
    
    # Cost Function and Optimizer
    # Cost = Mean Squared Error
    # Optimizer = Gradient Descent
    cost = tf.reduce_mean(tf.squared_difference(Y,output))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
    
    # Initialize all variables
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        print()
        print('Nodes number in hidden layer: ',n_hidden)
        sess.run(init)
        
        for step in range(epochs):
            cst_all = 0
            for i in range(len(train_X)):
                [op,cst] = sess.run([optimizer,cost],
                                     feed_dict={X:train_X[i:i+20],Y:train_Y[i:i+20]})
                
                cst_all += cst
                
            cst_all = cst_all/len(train_X)
            RMSE = tf.sqrt(tf.reduce_mean(tf.squared_difference(test_Y,
                                                 sess.run(output,feed_dict={X:test_X}))))
            
            if step%50 == 0:
                print("Epoch:%d, RMSE:%.5f, cost:%.5f"
                      %(step, sess.run(RMSE), cst_all))
        
        tvars = tf.trainable_variables()
        tvars_vals = sess.run(tvars)
        
        # Save the name of the variable alongside its value in a dictionary
        parameters = {var.name:val for var, val in zip(tvars, tvars_vals)}
        parameter_set.append(parameters)


Nodes number in hidden layer:  1
Epoch:0, RMSE:5.47377, cost:36.26152
Epoch:50, RMSE:5.50632, cost:25.10260
Epoch:100, RMSE:5.50764, cost:25.10469

Nodes number in hidden layer:  2
Epoch:0, RMSE:5.54996, cost:33.19550
Epoch:50, RMSE:5.50741, cost:25.09882
Epoch:100, RMSE:5.50784, cost:25.10031

Nodes number in hidden layer:  3
Epoch:0, RMSE:5.41994, cost:34.88515
Epoch:50, RMSE:5.50501, cost:25.10092
Epoch:100, RMSE:5.50604, cost:25.10274

Nodes number in hidden layer:  4
Epoch:0, RMSE:5.51146, cost:35.25474
Epoch:50, RMSE:5.50821, cost:25.10086
Epoch:100, RMSE:5.50793, cost:25.10323

Nodes number in hidden layer:  5
Epoch:0, RMSE:5.51502, cost:33.12718
Epoch:50, RMSE:5.50290, cost:25.09880
Epoch:100, RMSE:5.50339, cost:25.10041

Nodes number in hidden layer:  6
Epoch:0, RMSE:5.45617, cost:36.29402
Epoch:50, RMSE:5.50378, cost:25.10164
Epoch:100, RMSE:5.50406, cost:25.10375

Nodes number in hidden layer:  7
Epoch:0, RMSE:5.46046, cost:32.55410
Epoch:50, RMSE:5.50557, cost:25.09839
Epo

We can see that the lowest test RMSE occurs when n_hidden = 9. The weights and biases of that neural network model are as follows:

In [13]:
for key,val in parameter_set[8].items():
    print('%s\n'%key,val)

Weight1:0
 [[ 0.5146249   0.49574792  0.64518815  0.5259637  -0.65452456 -0.87737054
   0.19713464 -0.9427813  -0.21480758]
 [ 0.50300485  0.25255957 -0.21612377 -0.13811685 -0.20818281 -0.45342895
  -0.31308943 -0.68188983  0.40613502]
 [ 0.42019966 -0.22225115  0.12857616 -0.84278184 -0.84466887  0.61545116
   0.1266887   0.93728274 -0.17681837]]
Bias1:0
 [-1.8874505e-01 -4.2021748e-01 -2.3129682e-01 -6.1201084e-02
  0.0000000e+00 -9.6538372e-02  2.9072924e+00 -2.0517432e-04
  2.4742637e+00]
Weight2:0
 [[ 0.21458986]
 [-0.24147332]
 [-0.00415528]
 [ 0.0226226 ]
 [ 0.75259805]
 [-0.5844283 ]
 [ 2.4066591 ]
 [ 0.49551877]
 [ 2.0414302 ]]
Bias2:0
 [2.4787188]
